<font size="5">**Overlayed Spectrum**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from astropy.io import fits
import astropy.units as u
from specutils import Spectrum1D
from scipy import signal

def spectrum_overlay_polyfit(folder, date, constant=1.5):
    fits_spec_files = glob.glob(f"{folder}/*.fits")

    plt.figure(figsize=(12, 8))

    for i, file in enumerate(fits_spec_files):
        f = fits.open(file)

        #Retrieve flux
        flux = f[0].data * u.Unit('erg cm-2 s-1 AA-1')
        
        #Build the wavelength array
        init = f[0].header["CRVAL1"]
        refpix = f[0].header["CRPIX1"]
        delta = f[0].header["CDELT1"]
        npix = f[0].header["NAXIS1"]
        wl = (init + (np.arange(npix) + 1. - refpix) * delta) * u.AA

        #Fitting to get a horizontal spectrum
        spec=Spectrum1D(spectral_axis=wl, flux=flux)
        flux_fit_smooth=degrade_resolution(spec, 100.0).flux
        coeff=np.polyfit(wl.value, flux_fit_smooth, 4)
        poly=np.poly1d(coeff)
        flux_fitted= flux / poly(wl.value)
        
        #Smoothing
        spec = Spectrum1D(spectral_axis=wl, flux=flux_fitted)
        flux_smooth = degrade_resolution(spec, fwhm=10.0).flux
        flux_fitted += i*constant *u.Unit('erg cm-2 s-1 AA-1')
        flux_smooth += i * constant *u.Unit('erg cm-2 s-1 AA-1')

        label = " YSG"+os.path.basename(file)[3:-22].split('.')[0]  
        mask=(wl>3800*u.AA) & (wl<8700*u.AA)
        wl=wl[mask]
        plt.plot(wl, flux_fitted[mask], color="silver", linewidth=0.8, zorder=1)
        plt.plot(wl, flux_smooth[mask], color="black", linewidth=0.8, zorder=2)

        plt.text(wl[-1].value, flux_smooth[-1].value+0.5, label, fontsize=9, verticalalignment='center', color="black")

    plt.xlabel("Wavelength [$\\AA$]", fontsize=14)
    plt.xlim(3700, 9200)
    plt.ylim(-1, 37)
    plt.ylabel("Flux + Constant [$erg cm^{-2} s^{-1} Å^{-1}$]", fontsize=14)
    
    #Telluric absorption
    plt.axvspan(6850, 6950, color='grey', alpha=0.05)
    plt.axvspan(7150, 7400, color='grey', alpha=0.05)
    plt.axvspan(7590, 7700, color='grey', alpha=0.05)
    
    lines = {
    3835: {"line": "H9"}, 3889: {"line": "H8"}, 3933: {"line": "Ca II"}, 3970: {"line": r"$H\epsilon$"}, 4009: {"line": ""},
    4026: {"line": "He I"}, 4101: {"line": r"$H\delta$"}, 4120: {"line": ""}, 4144: {"line": "He I"}, 4200: {"line": "He II"},
    4340: {"line": r"H$\gamma$"}, 4387: {"line": "He I"}, 4471: {"line": "He I"},
    #4481: {"line": "Mg II"},
    4542: {"line": "He II"},
    #4552: {"line": "Si III"}, #4567: {"line": "Si III"}, #4574: {"line": "Si III"},
    4686: {"line": "He II"}, 4713: {"line": "He I"}, 4861: {"line": r"H$\beta$"}, 4922: {"line": "He I"}, 5016: {"line": "He I"},
    5317: {"line": "Fe II"}, 
    5577: {"line": "O I"},    #
    6300: {"line": "O I"},    #Triple
    6364: {"line": "O I"},    #
    5890: {"line": "Na I"},   #Double
    5896: {"line": ""},   #
    6563: {"line": r"$H\alpha$"}, 6678: {"line": "He I"}, 7065: {"line": "He I"}, 7291: {"line": ""}, 7323: {"line": "Ca II"},
    7771: {"line": "O I"}, 8446: {"line": "O I"}, 8498: {"line": ""}, 8542: {"line": "Ca II"}, 8662: {"line": ""} 
    }
    elevated_lines = {3835, 3889, 3970, 4101, 4861}
    
    for i, (lwl, prop) in enumerate(lines.items()):
        if lwl in elevated_lines:
            offset = 0.102
            ymax = 0.95
        elif lwl==4686:
            offset=0.11
            ymax=0.95
        else:
            offset = 0.054
            ymax = 0.90
        plt.axvline(x=lwl, color="grey", linestyle="--", lw=0.5, ymin=0, ymax=ymax)
        if prop["line"]=="He II" or prop["line"]=="Ca II":
            plt.text(lwl, max(flux_smooth.value)*(1+offset)+1.3, prop["line"], color="grey", fontsize=10, rotation=90, verticalalignment='top', horizontalalignment='center')
        else:
            plt.text(lwl, max(flux_smooth.value)*(1+offset)+1.1, prop["line"], color="grey", fontsize=10, rotation=90, verticalalignment='top', horizontalalignment='center')
        
    #Calcium
    plt.plot([3933, 3968], [0.593, 0.593], color="grey", linestyle="--", lw=0.5)
    plt.plot([8498, 8662], [0.593, 0.593], color="grey", linestyle="--", lw=0.5)
    
    plt.tight_layout()
    plt.savefig(f"{os.path.dirname(fits_spec_files[0])}/spectrum_overlay_{date}.png", dpi=150, bbox_inches='tight')
    plt.show()

<font size="5">**Sample of lightcurves and spectrum for TFG**

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from specutils import Spectrum1D
import matplotlib.ticker as ticker

def plot_sample(lightcurve_file, spectrum_file):
    fig = plt.figure(figsize=(6, 6))
    
    #Lighcurve
    ax1 = plt.subplot2grid((2, 1), (0, 0), fig=fig)
    df = pd.read_csv(lightcurve_file)
    
    df["date"] = df.iloc[:, 3]
    mag = df.iloc[:, 4]
    filtercode = df.iloc[:, 7]
    
    filter_colors = {"zg": "limegreen", "zr": "tomato", "zi": "dodgerblue"}
    filter_markers = {"zg": '^', "zi": '*', "zr": 'o'}
        
    for f in filtercode.unique():
        mask = filtercode == f
        color = filter_colors.get(f, "gray")
        marker = filter_markers.get(f, 'o')
        ax1.scatter(df["date"][mask], mag[mask], color=color, label=f, alpha=0.7, marker=marker)

    ax1.axvline(x=60251, color='black', linewidth=1, ymin=0.05, ymax=0.1)
    y_pos = ax1.get_ylim()[0] + 0.97 * (ax1.get_ylim()[1] - ax1.get_ylim()[0])
    ax1.text(60221, y_pos, "s", color='black', fontsize=12, ha='right', va='center')
    ax1.invert_yaxis()
    y_min, y_max = mag.min(), mag.max()
    ax1.set_ylim(y_max + 0.1, y_min - 0.1)
    ax1.set_xlabel("MJD")
    ax1.set_ylabel("Apparent magnitude (mag)")
    ax1.xaxis.set_label_position('top')
    ax1.xaxis.tick_top()
    ax1.legend(title="Filter")




    
    #Spectrum
    ax2 = plt.subplot2grid((2, 1), (1, 0), fig=fig)
    
    f = fits.open(spectrum_file)
    flux = f[0].data * u.Unit('erg cm-2 s-1 AA-1')
    
    init = f[0].header["CRVAL1"]
    refpix = f[0].header["CRPIX1"]
    delta = f[0].header["CDELT1"]
    npix = f[0].header["NAXIS1"]
    wl = (init + (np.arange(npix) + 1. - refpix) * delta) * u.AA
    spec = Spectrum1D(spectral_axis=wl, flux=flux)

    #Fitting to get a horizontal spectrum
    sigma1, flux_fit_smooth_ = degrade_resolution(spec, 100.0)
    flux_fit_smooth=flux_fit_smooth_.flux
    coeff=np.polyfit(wl.value, flux_fit_smooth, 3)
    poly=np.poly1d(coeff)
    flux_divided = flux / poly(wl.value)

    flux_divided=flux_divided-np.min(flux_divided)
    
    spec=Spectrum1D(spectral_axis=wl, flux=flux_divided)
    sigma2, flux_smooth_=degrade_resolution(spec, 5.0)
    flux_smooth=flux_smooth_.flux
    mask= (wl<7500*u.AA) & (wl>3800*u.AA)
    wl=wl[mask]
    ax2.plot(wl, np.log(flux_smooth[mask].value), linewidth=0.7, color="black", zorder=2)  
    ax2.plot(wl, np.log(flux_divided[mask].value), linewidth=0.6, color="lightgrey", zorder=1)

    lines = {
    3933: {"line": "Ca II"}, 3970: {"line": r"$H\epsilon$"}, 4009: {"line": ""},
    4026: {"line": "He I"}, 4101: {"line": r"$H\delta$"}, 4120: {"line": ""}, 4144: {"line": "He I"}, 4200: {"line": "He II"},
    4340: {"line": r"H$\gamma$"}, 4387: {"line": "He I"}, 4471: {"line": "He I"},
    #4481: {"line": "Mg II"},
    4542: {"line": "He II"},
    #4552: {"line": "Si III"}, #4567: {"line": "Si III"}, #4574: {"line": "Si III"},
    4686: {"line": "He II"}, 4713: {"line": "He I"}, 4861: {"line": r"H$\beta$"}, 4922: {"line": "He I"}, 5016: {"line": "He I"},
    #5317: {"line": "Fe II"}, 
    5577: {"line": "O I"},    #
    6300: {"line": "O I"},    #Triple
    6364: {"line": "O I"},    #
    5890: {"line": "Na I"},   #Double
    5896: {"line": ""},   #
    6563: {"line": r"$H\alpha$"}, 6678: {"line": "He I"}, 7065: {"line": "He I"}, 7291: {"line": ""}, 7323: {"line": "Ca II"},
    }
    non_elevated= {3933, 4120, 4144, 4686, 4861}
    
    for i, (lwl, prop) in enumerate(lines.items()):
        if lwl in non_elevated:
            offset = 0.2
            ymax = 0.75
        else:
            offset = 0
            ymax = 0.85
        plt.axvline(x=lwl, color="grey", linestyle="--", lw=0.5, ymin=0., ymax=ymax)
        plt.text(lwl, max(flux_smooth.value)/3+0.3-offset, prop["line"], color="grey", fontsize=8, rotation=90, verticalalignment='top', horizontalalignment='center')
    
    ax2.yaxis.get_offset_text().set_visible(False)
    ax2.set_xlabel("Wavelength [$\\AA$]")
    ax2.set_ylabel("Normalized Flux")
    ax2.set_xlim(3780, 7510)
    ax2.set_ylim(-0.49, 1.55) 
    #plt.yscale("log")
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.0)
    plt.savefig("lc_spectrum_plot.png", dpi=150, bbox_inches='tight')
    plt.show()